In [1]:
# ATSIT 2019: dimensional speech emotion recognition from text feature

# uncomment these to run on CPU only
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
import matplotlib.pyplot as plt
import pickle
import keras.backend as K
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, LSTM, TimeDistributed, Bidirectional, Embedding, Dropout, Flatten, concatenate, Convolution1D
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

from keras.callbacks import EarlyStopping
import codecs

import io
import random as rn
import tensorflow as tf

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')

# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# load fastext model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))

# load model
EMBEDDING_DIM = 300
file_loc = '../../dimensionalSM/data/glove.42B.300d.txt'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

# split train/test
split = 7869
earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)



Found 3437 unique tokens
../../dimensionalSM/data/glove.42B.300d.txt
G Word embeddings: 1917494
G Null word embeddings: 128


In [4]:
# model: GRU
def text_model1():
    inputs = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(inputs)
    #net = Bidirectional(LSTM(64, return_sequences=True))(net)
#     net = Bidirectional(LSTM(64, return_sequences=True))(net)
#     net = Bidirectional(LSTM(64, return_sequences=True))(net)
    net = Bidirectional(LSTM(128, return_sequences=True))(net)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    out_text = Dense(64)(net)
    #out_text = Dropout(0.5)(net)
    target_names = ('v', 'a', 'd')
    outputs = [Dense(1, name=name)(out_text) for name in target_names]

    model = Model(inputs=inputs, outputs=outputs) #=[out1, out2, out3])
    model.compile(loss=ccc_loss, #{'v': ccc_loss, 'a': ccc_loss, 'd': ccc_loss},
                     loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model
model = text_model1()
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     1031400     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 100, 256)     439296      embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 128)          164352      bidirectional_1[0][0]            
___________

In [5]:
# main function for for LOSO
def main(time):
    model = text_model1()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model.fit( x_train_text[:8000], 
                      vad[:8000].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model.evaluate( x_train_text[8000:], vad[8000:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    weight = []
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('text_blstm8000_no_dropout.csv', header=['time','v','a','d', 'ave'])



Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 43s 7ms/step - loss: 1.8738 - v_loss: 0.5987 - a_loss: 0.6370 - d_loss: 0.6382 - v_ccc: 0.4013 - a_ccc: 0.3630 - d_ccc: 0.3618 - val_loss: 1.9930 - val_v_loss: 0.7148 - val_a_loss: 0.6462 - val_d_loss: 0.6320 - val_v_ccc: 0.2852 - val_a_ccc: 0.3538 - val_d_ccc: 0.3680
Epoch 2/50
6400/6400 [==============================] - 40s 6ms/step - loss: 1.4506 - v_loss: 0.4075 - a_loss: 0.5265 - d_loss: 0.5166 - v_ccc: 0.5925 - a_ccc: 0.4735 - d_ccc: 0.4834 - val_loss: 1.9844 - val_v_loss: 0.6849 - val_a_loss: 0.6248 - val_d_loss: 0.6747 - val_v_ccc: 0.3151 - val_a_ccc: 0.3752 - val_d_ccc: 0.3253
Epoch 3/50
6400/6400 [==============================] - 40s 6ms/step - loss: 1.2634 - v_loss: 0.3365 - a_loss: 0.4710 - d_loss: 0.4558 - v_ccc: 0.6635 - a_ccc: 0.5290 - d_ccc: 0.5442 - val_loss: 1.9288 - val_v_loss: 0.6157 - val_a_loss: 0.6225 - val_d_loss: 0.6906 - val_v_ccc: 0.3843 - val_a_ccc: 0.3

Epoch 9/50
6400/6400 [==============================] - 44s 7ms/step - loss: 0.7155 - v_loss: 0.1730 - a_loss: 0.2725 - d_loss: 0.2700 - v_ccc: 0.8270 - a_ccc: 0.7275 - d_ccc: 0.7300 - val_loss: 1.9119 - val_v_loss: 0.5791 - val_a_loss: 0.6311 - val_d_loss: 0.7017 - val_v_ccc: 0.4209 - val_a_ccc: 0.3689 - val_d_ccc: 0.2983
Epoch 10/50
6400/6400 [==============================] - 44s 7ms/step - loss: 0.6596 - v_loss: 0.1565 - a_loss: 0.2532 - d_loss: 0.2499 - v_ccc: 0.8435 - a_ccc: 0.7468 - d_ccc: 0.7501 - val_loss: 1.9138 - val_v_loss: 0.5793 - val_a_loss: 0.6469 - val_d_loss: 0.6876 - val_v_ccc: 0.4207 - val_a_ccc: 0.3531 - val_d_ccc: 0.3124
Epoch 11/50
6400/6400 [==============================] - 44s 7ms/step - loss: 0.6048 - v_loss: 0.1460 - a_loss: 0.2345 - d_loss: 0.2243 - v_ccc: 0.8540 - a_ccc: 0.7655 - d_ccc: 0.7757 - val_loss: 1.9296 - val_v_loss: 0.6183 - val_a_loss: 0.6475 - val_d_loss: 0.6637 - val_v_ccc: 0.3817 - val_a_ccc: 0.3525 - val_d_ccc: 0.3363
Epoch 12/50
6400/6400 [

2039/2039 [==============================] - 4s 2ms/step
[0.4081624746322632, 0.3662606179714203, 0.3962591886520386]
2
Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 46s 7ms/step - loss: 1.8717 - v_loss: 0.5946 - a_loss: 0.6457 - d_loss: 0.6314 - v_ccc: 0.4054 - a_ccc: 0.3543 - d_ccc: 0.3686 - val_loss: 1.9998 - val_v_loss: 0.6824 - val_a_loss: 0.6554 - val_d_loss: 0.6620 - val_v_ccc: 0.3176 - val_a_ccc: 0.3446 - val_d_ccc: 0.3380
Epoch 2/50
6400/6400 [==============================] - 43s 7ms/step - loss: 1.4431 - v_loss: 0.4030 - a_loss: 0.5234 - d_loss: 0.5167 - v_ccc: 0.5970 - a_ccc: 0.4766 - d_ccc: 0.4833 - val_loss: 1.9814 - val_v_loss: 0.6815 - val_a_loss: 0.6369 - val_d_loss: 0.6631 - val_v_ccc: 0.3185 - val_a_ccc: 0.3631 - val_d_ccc: 0.3369
Epoch 3/50
6400/6400 [==============================] - 43s 7ms/step - loss: 1.2672 - v_loss: 0.3391 - a_loss: 0.4684 - d_loss: 0.4596 - v_ccc: 0.6609 - a_ccc: 0.5316 - d_ccc: 0.5404

6400/6400 [==============================] - 43s 7ms/step - loss: 0.6014 - v_loss: 0.1465 - a_loss: 0.2292 - d_loss: 0.2257 - v_ccc: 0.8535 - a_ccc: 0.7708 - d_ccc: 0.7743 - val_loss: 1.8905 - val_v_loss: 0.5766 - val_a_loss: 0.6478 - val_d_loss: 0.6662 - val_v_ccc: 0.4234 - val_a_ccc: 0.3522 - val_d_ccc: 0.3338
Epoch 12/50
6400/6400 [==============================] - 43s 7ms/step - loss: 0.5619 - v_loss: 0.1345 - a_loss: 0.2146 - d_loss: 0.2129 - v_ccc: 0.8655 - a_ccc: 0.7854 - d_ccc: 0.7871 - val_loss: 1.8787 - val_v_loss: 0.5752 - val_a_loss: 0.6471 - val_d_loss: 0.6563 - val_v_ccc: 0.4248 - val_a_ccc: 0.3529 - val_d_ccc: 0.3437
Epoch 13/50
6400/6400 [==============================] - 41s 6ms/step - loss: 0.5158 - v_loss: 0.1277 - a_loss: 0.1945 - d_loss: 0.1936 - v_ccc: 0.8723 - a_ccc: 0.8055 - d_ccc: 0.8064 - val_loss: 1.9275 - val_v_loss: 0.5758 - val_a_loss: 0.6811 - val_d_loss: 0.6706 - val_v_ccc: 0.4242 - val_a_ccc: 0.3189 - val_d_ccc: 0.3294
Epoch 14/50
6400/6400 [===========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6400/6400 [==============================] - 43s 7ms/step - loss: 0.7858 - v_loss: 0.1894 - a_loss: 0.2964 - d_loss: 0.3001 - v_ccc: 0.8106 - a_ccc: 0.7036 - d_ccc: 0.6999 - val_loss: 1.8927 - val_v_loss: 0.5818 - val_a_loss: 0.6463 - val_d_loss: 0.6646 - val_v_ccc: 0.4182 - val_a_ccc: 0.3537 - val_d_ccc: 0.3354
Epoch 9/50
6400/6400 [==============================] - 44s 7ms/step - loss: 0.7126 - v_loss: 0.1712 - a_loss: 0.2721 - d_loss: 0.2693 - v_ccc: 0.8288 - a_ccc: 0.7279 - d_ccc: 0.7307 - val_loss: 2.0272 - val_v_loss: 0.5849 - val_a_loss: 0.6801 - val_d_loss: 0.7622 - val_v_ccc: 0.4151 - val_a_ccc: 0.3199 - val_d_ccc: 0.2378
Epoch 10/50
6400/6400 [==============================] - 43s 7ms/step - loss: 0.6522 - v_loss: 0.1542 - a_loss: 0.2472 - d_loss: 0.2508 - v_ccc: 0.8458 - a_ccc: 0.7528 - d_ccc: 0.7492 - val_loss: 1.8884 - val_v_loss: 0.5666 - val_a_loss: 0.6472 - val_d_loss: 0.6746 - val_v_ccc: 0.4334 - val_a_ccc: 0.3528 - val_d_ccc: 0.3254
Epoch 11/50
6400/6400 [============

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6400/6400 [==============================] - 43s 7ms/step - loss: 1.4469 - v_loss: 0.4031 - a_loss: 0.5272 - d_loss: 0.5165 - v_ccc: 0.5969 - a_ccc: 0.4728 - d_ccc: 0.4835 - val_loss: 1.9271 - val_v_loss: 0.6503 - val_a_loss: 0.6264 - val_d_loss: 0.6503 - val_v_ccc: 0.3497 - val_a_ccc: 0.3736 - val_d_ccc: 0.3497
Epoch 3/50
6400/6400 [==============================] - 44s 7ms/step - loss: 1.2668 - v_loss: 0.3377 - a_loss: 0.4703 - d_loss: 0.4588 - v_ccc: 0.6623 - a_ccc: 0.5297 - d_ccc: 0.5412 - val_loss: 1.8507 - val_v_loss: 0.6164 - val_a_loss: 0.6042 - val_d_loss: 0.6301 - val_v_ccc: 0.3836 - val_a_ccc: 0.3958 - val_d_ccc: 0.3699
Epoch 4/50
6400/6400 [==============================] - 43s 7ms/step - loss: 1.1365 - v_loss: 0.2903 - a_loss: 0.4290 - d_loss: 0.4172 - v_ccc: 0.7097 - a_ccc: 0.5710 - d_ccc: 0.5828 - val_loss: 1.8926 - val_v_loss: 0.6066 - val_a_loss: 0.6365 - val_d_loss: 0.6494 - val_v_ccc: 0.3934 - val_a_ccc: 0.3635 - val_d_ccc: 0.3506
Epoch 5/50
6400/6400 [==============

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

